In [1]:
import random
import math
import pygame
import time

pygame 2.6.1 (SDL 2.28.4, Python 3.12.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
#total ingredient lists
all_fruits = ["banana", "apple", "blueberry", "strawberry",
              "raspberry", "mango", "orange", "pineapple",
              "kiwi", "mixed berries", "watermelon", "cherry", "blackberry"]
all_liquids = ["cow's milk", "soy milk", "almond milk", "coconut milk",
               "oat milk", "rice milk", "hazelnut milk", "skim milk",
               "natural yoghurt", "apple juice", "orange juice", "lemon juice",
               "cranberry juice", "pineapple juice", "grape juice", "pomegranate juice"]
all_miscs = ["honey", "maple syrup", "oats", "seeds", "peanut butter",
             "almond butter", "protein powder", "mint", "cinnamon", "ginger", 
             "tumeric"]

#dietary restrictions
non_vegan = ["cow's milk", "skim milk", "natural yoghurt", "honey"]
dairy = ["cow's milk", "skim milk", "natural yoghurt"]

#starting ingredient lists
fruits = ["banana", "apple", "blueberry", "strawberry"]
liquids = ["cow's milk", "soy milk", "apple juice", "lemon juice"]
miscs = ["honey", "oats", "protein powder", "peanut butter"]

#starting max fruits
max_fruits = 2
#starting max liquids
max_liquids = 1
#starting max other ingredients
max_miscs = 1

In [3]:
#create order with up to 2 fruits, 1 liquid, 1 other
def create_order():
    order_ingredients = []
    #random liquid
    for liquid in random.sample(liquids, random.randint(1, max_liquids)):
        order_ingredients.append(liquid)
    #random misc
    for misc in random.sample(miscs, random.randint(1, max_miscs)):
        order_ingredients.append(misc)
    #get multiple fruits
    for fruit in random.sample(fruits, random.randint(1, max_fruits)):
        order_ingredients.append(fruit)

    return order_ingredients
    
create_order()

['soy milk', 'peanut butter', 'apple', 'banana']

In [4]:
# initialize pygame
pygame.init()

#screen setup
WIDTH = 960
HEIGHT = 540

#create screen
screen = pygame.display.set_mode((WIDTH, HEIGHT))

#create clock
clock = pygame.time.Clock

#title & icon
pygame.display.set_caption("Smoothie Shop")
icon = pygame.image.load("art/smoothie.png")
pygame.display.set_icon(icon)
# <a href="https://www.flaticon.com/free-icons/smoothie" title="smoothie icons">Smoothie icons created by Flat Icons - Flaticon</a>

#font
pygame.font.init()
main_font = pygame.font.SysFont('Kristen ITC', 28)
time_font = pygame.font.SysFont('LT Stopwatch', 60)

#load background images
bar = pygame.image.load("art/bar.png").convert_alpha()
prep_area = pygame.image.load("art/prep_area.png").convert_alpha()

#load blender images
blender = pygame.image.load("art/blender.png").convert_alpha()
mixture = pygame.image.load("art/blender_contents.png").convert_alpha()

#load button images
kitchen_button_img = pygame.image.load("art/kitchen_button.png").convert_alpha()
service_button_img = pygame.image.load("art/service_button.png").convert_alpha()

#button class
class Button():
    def __init__(self, x, y, image, scale):
          width = image.get_width()
          height = image.get_height()
          self.image = pygame.transform.scale(image, (int(width*scale),int(height*scale)))
          self.rect = self.image.get_rect()
          self.rect.topleft = (x, y)
    def draw(self):
         screen.blit(self.image, (self.rect.x, self.rect.y))

#create button instances
kitchen_button = Button(50, HEIGHT-125, kitchen_button_img, 4)
service_button = Button(WIDTH-178, HEIGHT-125, service_button_img, 4)

#game loop
running = True
while running:

    screen.fill((255, 120, 180)) #fill pink
    
    kitchen_button.draw()
    service_button.draw()
      
    for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

    pygame.display.update()

pygame.quit()